In [50]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import os
import pickle

In [5]:
iurl = "https://itch.io/games"
turl = "https://itch.io/tags"

In [30]:
def gamelist(url):

    r = requests.get(url)
    rt = r.text
    # print(rt)
    soup = bs(rt, "lxml")
    lclass = "title game_link"
    allanc = soup.find_all("a", class_=lclass)
    # print(allanc)
    lset = set()
    for anc in allanc:
        lset.add(anc.get_text())
    return list(lset)

def taglist(url):
    # note this is for "top tags", not sure what the cutoff is
    r = requests.get(url)
    rt = r.text
    # print(rt)
    soup = bs(rt, "lxml")
    tclass = "outline_button"
    allanc = soup.find_all("a", class_=tclass)
    # print(allanc)
    tset = set()
    for anc in allanc:
        txt = anc.get_text()
        href = anc['href']
        lab = anc['data-label']
        ttup = (txt, href, lab)
        tset.add(ttup)
    return list(tset)

def tagscrape(taglist):
    tbase = "https://itch.io"
    tclist = list()
    for tag in taglist:
        turl = tbase + tag[1]
        r = requests.get(turl)
        rt = r.text
        # print(rt)
        soup = bs(rt, "lxml")
        cclass = "game_count"
        gcount = soup.find("nobr", class_=cclass)
        gctxt = gcount.get_text()
        gcdig = u''.join(c for c in gctxt if '0' <= c <= '9')
        gctup = (tag[0], tag[1], gcdig, tag[2])
        tclist.append(gctup)
    return tclist

def gamelist_from_file(file):
    with open (file, 'r') as fh:
        rt = fh.read()
    soup = bs(rt, "lxml")
    lclass = "title game_link"
    allanc = soup.find_all("a", class_=lclass)
    # print(allanc)
    lset = set()
    for anc in allanc:
        txt = anc.get_text()
        href = anc['href']
        lab = anc['data-label']
        gid = lab.split(':')
        ltup = (txt, href, int(gid[1]))
        lset.add(ltup)
    return list(lset)

In [19]:
# example url: <a class="title game_link" href="https://owch.itch.io/btab" data-label="game:231714:title" data-action="game_grid">BEAT THE ART BREAKER [DEMO]</a>

itchtags = taglist(turl)
# itchlist = gamelist(iurl)

# https://itch.io:443/games/tag-horrible
# https://itch.io:443/games/tag-horrible?page=2

[('MIDI', '/games/tag-midi', 'midi'),
 ('norway', '/games/tag-norway', 'norway')]

In [33]:
tslist = tagscrape(itchtags)

In [34]:
tslist

[('Survival Horror', '/games/tag-survival-horror', '664', 'survival-horror'),
 ('1-bit', '/games/tag-1-bit', '489', '1-bit'),
 ('Time Travel', '/games/tag-time-travel', '212', 'time-travel'),
 ('War', '/games/tag-war', '523', 'war'),
 ('LGBT', '/games/tag-lgbt', '505', 'lgbt'),
 ('Survival', '/games/tag-survival', '2257', 'survival'),
 ('Leap Motion', '/games/tag-leap-motion', '264', 'leap-motion'),
 ('Racing', '/games/tag-racing', '1200', 'racing'),
 ('Score Attack', '/games/tag-score-attack', '665', 'score-attack'),
 ('Ludum Dare 36', '/games/tag-ludum-dare-36', '665', 'ludum-dare-36'),
 ('Manga', '/games/tag-manga', '39', 'manga'),
 ('Dark Humor', '/games/tag-dark-humor', '475', 'dark-humor'),
 ('tablet', '/games/tag-tablet', '161', 'tablet'),
 ('typing', '/games/tag-typing', '216', 'typing'),
 ('Visual Novel', '/games/genre-visual-novel', '2517', 'visual-novel'),
 ('nature', '/games/tag-nature', '260', 'nature'),
 ('Farming', '/games/tag-farming', '177', 'farming'),
 ('one-hit-kill

In [40]:
# get my curl on
# looks like games end around page 3466

istart = 2000
# iend = 2345
ichunk = 100
irep = 15

for rep in range(irep):
    
    rstart = (rep * ichunk) + istart
    rend = (((rep + 1) * ichunk) + istart) - 1
    srstart = str(rstart)
    srend = str(rend)
    prep = "rsrart: {} rend: {}".format(srstart, srend)
    print(prep)
    curl = "https://itch.io:443/games?page=[{}-{}]".format(srstart, srend)
    cfile = "html/big_itch_{}_{}.html".format(srstart, srend)

    !curl $curl >> $cfile

rsrart: 2000 rend: 2099

[1/100]: https://itch.io:443/games?page=2000 --> <stdout>
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 64906    0 64906    0     0  64906      0 --:--:-- --:--:-- --:--:--  243k

[2/100]: https://itch.io:443/games?page=2001 --> <stdout>
100 64876    0 64876    0     0  64876      0 --:--:-- --:--:-- --:--:--  450k

[3/100]: https://itch.io:443/games?page=2002 --> <stdout>
100 67789    0 67789    0     0  67789      0 --:--:-- --:--:-- --:--:-- 67789

[4/100]: https://itch.io:443/games?page=2003 --> <stdout>
100 66557    0 66557    0     0  66557      0 --:--:-- --:--:-- --:--:-- 66557

[5/100]: https://itch.io:443/games?page=2004 --> <stdout>
100 64784    0 64784    0     0  64784      0 --:--:-- --:--:-- --:--:--  645k

[6/100]: https://itch.io:443/games?page=2005 --> <stdout>
100 65367    0 65367    0     0  65367      0 --:--:-- --:--:-- --:--

100 66075    0 66075    0     0  66075      0 --:--:-- --:--:-- --:--:--  384k

[59/100]: https://itch.io:443/games?page=2058 --> <stdout>
100 66168    0 66168    0     0  66168      0 --:--:-- --:--:-- --:--:-- 66168

[60/100]: https://itch.io:443/games?page=2059 --> <stdout>
100 67025    0 67025    0     0  67025      0 --:--:-- --:--:-- --:--:-- 67025

[61/100]: https://itch.io:443/games?page=2060 --> <stdout>
100 66736    0 66736    0     0  66736      0 --:--:-- --:--:-- --:--:--  307k

[62/100]: https://itch.io:443/games?page=2061 --> <stdout>
100 65695    0 65695    0     0  65695      0 --:--:-- --:--:-- --:--:-- 65695

[63/100]: https://itch.io:443/games?page=2062 --> <stdout>
100 65718    0 65718    0     0  65718      0 --:--:-- --:--:-- --:--:--  583k

[64/100]: https://itch.io:443/games?page=2063 --> <stdout>
100 65319    0 65319    0     0  65319      0 --:--:-- --:--:-- --:--:-- 65319

[65/100]: https://itch.io:443/games?page=2064 --> <stdout>
100 66090    0 66090    0  

100 65450    0 65450    0     0  65450      0 --:--:-- --:--:-- --:--:-- 65450

[17/100]: https://itch.io:443/games?page=2116 --> <stdout>
100 63880    0 63880    0     0  63880      0 --:--:-- --:--:-- --:--:--  321k

[18/100]: https://itch.io:443/games?page=2117 --> <stdout>
100 64344    0 64344    0     0  64344      0 --:--:-- --:--:-- --:--:-- 64344

[19/100]: https://itch.io:443/games?page=2118 --> <stdout>
100 65928    0 65928    0     0  65928      0 --:--:-- --:--:-- --:--:--  270k

[20/100]: https://itch.io:443/games?page=2119 --> <stdout>
100 66073    0 66073    0     0  66073      0 --:--:-- --:--:-- --:--:--  700k

[21/100]: https://itch.io:443/games?page=2120 --> <stdout>
100 65122    0 65122    0     0  65122      0 --:--:-- --:--:-- --:--:-- 65122

[22/100]: https://itch.io:443/games?page=2121 --> <stdout>
100 65774    0 65774    0     0  65774      0 --:--:-- --:--:-- --:--:-- 16.2M

[23/100]: https://itch.io:443/games?page=2122 --> <stdout>
100 67501    0 67501    0  

100 64604    0 64604    0     0  64604      0 --:--:-- --:--:-- --:--:--  553k

[76/100]: https://itch.io:443/games?page=2175 --> <stdout>
100 65264    0 65264    0     0  65264      0 --:--:-- --:--:-- --:--:-- 65264

[77/100]: https://itch.io:443/games?page=2176 --> <stdout>
100 64206    0 64206    0     0  64206      0 --:--:-- --:--:-- --:--:--  597k

[78/100]: https://itch.io:443/games?page=2177 --> <stdout>
100 64158    0 64158    0     0  64158      0 --:--:-- --:--:-- --:--:-- 64158

[79/100]: https://itch.io:443/games?page=2178 --> <stdout>
100 63296    0 63296    0     0  63296      0 --:--:-- --:--:-- --:--:--  383k

[80/100]: https://itch.io:443/games?page=2179 --> <stdout>
100 63626    0 63626    0     0  63626      0 --:--:-- --:--:-- --:--:-- 63626

[81/100]: https://itch.io:443/games?page=2180 --> <stdout>
100 64766    0 64766    0     0  64766      0 --:--:-- --:--:-- --:--:--  305k

[82/100]: https://itch.io:443/games?page=2181 --> <stdout>
100 65243    0 65243    0  

100 69345    0 69345    0     0  69345      0 --:--:-- --:--:-- --:--:--  368k

[34/100]: https://itch.io:443/games?page=2233 --> <stdout>
100 68712    0 68712    0     0  68712      0 --:--:-- --:--:-- --:--:-- 68712

[35/100]: https://itch.io:443/games?page=2234 --> <stdout>
100 69244    0 69244    0     0  69244      0 --:--:-- --:--:-- --:--:--  402k

[36/100]: https://itch.io:443/games?page=2235 --> <stdout>
100 69455    0 69455    0     0  69455      0 --:--:-- --:--:-- --:--:-- 69455

[37/100]: https://itch.io:443/games?page=2236 --> <stdout>
100 68511    0 68511    0     0  68511      0 --:--:-- --:--:-- --:--:--  327k

[38/100]: https://itch.io:443/games?page=2237 --> <stdout>
100 69350    0 69350    0     0  69350      0 --:--:-- --:--:-- --:--:-- 69350

[39/100]: https://itch.io:443/games?page=2238 --> <stdout>
100 69208    0 69208    0     0  69208      0 --:--:-- --:--:-- --:--:-- 4013k

[40/100]: https://itch.io:443/games?page=2239 --> <stdout>
100 68610    0 68610    0  

...

100 65701    0 65701    0     0  65701      0 --:--:-- --:--:-- --:--:-- 65701

[25/100]: https://itch.io:443/games?page=3224 --> <stdout>
100 66190    0 66190    0     0  66190      0 --:--:-- --:--:-- --:--:--  710k

[26/100]: https://itch.io:443/games?page=3225 --> <stdout>
100 65727    0 65727    0     0  65727      0 --:--:-- --:--:-- --:--:-- 65727

[27/100]: https://itch.io:443/games?page=3226 --> <stdout>
100 65627    0 65627    0     0  65627      0 --:--:-- --:--:-- --:--:--  613k

[28/100]: https://itch.io:443/games?page=3227 --> <stdout>
100 65091    0 65091    0     0  65091      0 --:--:-- --:--:-- --:--:-- 65091

[29/100]: https://itch.io:443/games?page=3228 --> <stdout>
100 65273    0 65273    0     0  65273      0 --:--:-- --:--:-- --:--:-- 65273

[30/100]: https://itch.io:443/games?page=3229 --> <stdout>
100 65433    0 65433    0     0  65433      0 --:--:--  0:00:01 --:--:-- 61209

[31/100]: https://itch.io:443/games?page=3230 --> <stdout>
100 65750    0 65750    0  

100 64940    0 64940    0     0  64940      0 --:--:-- --:--:-- --:--:-- 64940

[84/100]: https://itch.io:443/games?page=3283 --> <stdout>
100 65517    0 65517    0     0  65517      0 --:--:-- --:--:-- --:--:--  515k

[85/100]: https://itch.io:443/games?page=3284 --> <stdout>
100 64742    0 64742    0     0  64742      0 --:--:-- --:--:-- --:--:-- 64742

[86/100]: https://itch.io:443/games?page=3285 --> <stdout>
100 65102    0 65102    0     0  65102      0 --:--:-- --:--:-- --:--:--  287k

[87/100]: https://itch.io:443/games?page=3286 --> <stdout>
100 64650    0 64650    0     0  64650      0 --:--:-- --:--:-- --:--:-- 64650

[88/100]: https://itch.io:443/games?page=3287 --> <stdout>
100 64832    0 64832    0     0  64832      0 --:--:-- --:--:-- --:--:--  217k

[89/100]: https://itch.io:443/games?page=3288 --> <stdout>
100 66271    0 66271    0     0  66271      0 --:--:-- --:--:-- --:--:-- 66271

[90/100]: https://itch.io:443/games?page=3289 --> <stdout>
100 65572    0 65572    0  

100 66016    0 66016    0     0  66016      0 --:--:-- --:--:-- --:--:-- 66016

[42/100]: https://itch.io:443/games?page=3341 --> <stdout>
100 66522    0 66522    0     0  66522      0 --:--:-- --:--:-- --:--:--  224k

[43/100]: https://itch.io:443/games?page=3342 --> <stdout>
100 65425    0 65425    0     0  65425      0 --:--:-- --:--:-- --:--:--  317k

[44/100]: https://itch.io:443/games?page=3343 --> <stdout>
100 66319    0 66319    0     0  66319      0 --:--:-- --:--:-- --:--:-- 66319

[45/100]: https://itch.io:443/games?page=3344 --> <stdout>
100 65391    0 65391    0     0  65391      0 --:--:-- --:--:-- --:--:--  271k

[46/100]: https://itch.io:443/games?page=3345 --> <stdout>
100 65850    0 65850    0     0  65850      0 --:--:-- --:--:-- --:--:-- 65850

[47/100]: https://itch.io:443/games?page=3346 --> <stdout>
100 65721    0 65721    0     0  65721      0 --:--:-- --:--:-- --:--:--  268k

[48/100]: https://itch.io:443/games?page=3347 --> <stdout>
100 63870    0 63870    0  

100 65040    0 65040    0     0  65040      0 --:--:-- --:--:-- --:--:-- 65040
rsrart: 3400 rend: 3499

[1/100]: https://itch.io:443/games?page=3400 --> <stdout>
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 65617    0 65617    0     0  65617      0 --:--:-- --:--:-- --:--:--  120k

[2/100]: https://itch.io:443/games?page=3401 --> <stdout>
100 65397    0 65397    0     0  65397      0 --:--:-- --:--:-- --:--:-- 65397

[3/100]: https://itch.io:443/games?page=3402 --> <stdout>
100 64757    0 64757    0     0  64757      0 --:--:-- --:--:-- --:--:--  343k

[4/100]: https://itch.io:443/games?page=3403 --> <stdout>
100 64336    0 64336    0     0  64336      0 --:--:-- --:--:-- --:--:-- 64336

[5/100]: https://itch.io:443/games?page=3404 --> <stdout>
100 64450    0 64450    0     0  64450      0 --:--:-- --:--:-- --:--:--  295k

[6/100]: https://itch.io:443/games?page=3405 -->

100 63815    0 63815    0     0  63815      0 --:--:-- --:--:-- --:--:--  224k

[58/100]: https://itch.io:443/games?page=3457 --> <stdout>
100 65133    0 65133    0     0  65133      0 --:--:-- --:--:-- --:--:--  681k

[59/100]: https://itch.io:443/games?page=3458 --> <stdout>
100 65442    0 65442    0     0  65442      0 --:--:-- --:--:-- --:--:-- 65442

[60/100]: https://itch.io:443/games?page=3459 --> <stdout>
100 65707    0 65707    0     0  65707      0 --:--:-- --:--:-- --:--:--  384k

[61/100]: https://itch.io:443/games?page=3460 --> <stdout>
100 63123    0 63123    0     0  63123      0 --:--:-- --:--:-- --:--:-- 63123

[62/100]: https://itch.io:443/games?page=3461 --> <stdout>
100 64430    0 64430    0     0  64430      0 --:--:-- --:--:-- --:--:--  361k

[63/100]: https://itch.io:443/games?page=3462 --> <stdout>
100 62666    0 62666    0     0  62666      0 --:--:-- --:--:-- --:--:-- 62666

[64/100]: https://itch.io:443/games?page=3463 --> <stdout>
100 63193    0 63193    0  

In [48]:
htmdir = "html"

htmlist = list()
for file in os.listdir(htmdir):
    if file[-5:] == '.html':
        htmlist.append(file)

bilist = list()
for itchfile in htmlist:
    fpath = "html/" + itchfile
    sublist = gamelist_from_file(fpath)
    bilist.extend(sublist)
    
len(bilist)

104666

In [52]:
with open('all_games.pkl','wb') as fh:
    pickle.dump(bilist,fh)

In [73]:
for item in bilist[25:200]:
    curl = item[1]
    cfile = "html/game_pages/{}.html".format(item[2])

    !curl $curl > $cfile

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17727    0 17727    0     0  17727      0 --:--:-- --:--:-- --:--:-- 92811
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14205    0 14205    0     0  14205      0 --:--:-- --:--:-- --:--:-- 56820
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14523    0 14523    0     0  14523      0 --:--:-- --:--:-- --:--:--  100k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17916    0 17916    0     0  17916      0 --:--:-- --:--:-- --:--:--  130k
  % Total    % Received % Xferd  Average Speed   Tim

100 15217    0 15217    0     0  15217      0 --:--:-- --:--:-- --:--:-- 36059
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17267    0 17267    0     0  17267      0 --:--:-- --:--:-- --:--:--  103k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16777    0 16777    0     0  16777      0 --:--:-- --:--:-- --:--:-- 56110
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15317    0 15317    0     0  15317      0 --:--:-- --:--:-- --:--:-- 35871
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15184    0 15184    0     0  15184      0 --:--:

100 19373    0 19373    0     0  19373      0 --:--:-- --:--:-- --:--:-- 96865
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16022    0 16022    0     0  16022      0 --:--:-- --:--:-- --:--:-- 92612
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16524    0 16524    0     0  16524      0 --:--:-- --:--:-- --:--:-- 79442
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17102    0 17102    0     0  17102      0 --:--:-- --:--:-- --:--:-- 82618
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14378    0 14378    0     0  14378      0 --:--:

100 19012    0 19012    0     0  19012      0 --:--:-- --:--:-- --:--:--  119k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15611    0 15611    0     0  15611      0 --:--:-- --:--:-- --:--:--   98k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16500    0 16500    0     0  16500      0 --:--:-- --:--:-- --:--:-- 97058
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13915    0 13915    0     0  13915      0 --:--:-- --:--:-- --:--:-- 84847
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14582    0 14582    0     0  14582      0 --:--:

100 17832    0 17832    0     0  17832      0 --:--:-- --:--:-- --:--:-- 67545
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17522    0 17522    0     0  17522      0 --:--:-- --:--:-- --:--:--  118k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15781    0 15781    0     0  15781      0 --:--:-- --:--:-- --:--:-- 24930
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15064    0 15064    0     0  15064      0 --:--:-- --:--:-- --:--:-- 92987
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13558    0 13558    0     0  13558      0 --:--:

100 16426    0 16426    0     0  16426      0 --:--:-- --:--:-- --:--:--  115k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13451    0 13451    0     0  13451      0 --:--:-- --:--:-- --:--:-- 34578


In [62]:
columns = ['name', 'url', 'gid']

df = pd.DataFrame([g for g in bilist], columns=columns)

# print df

df = df.set_index('gid')
#if you need set column name
df.columns.name = 'game_id'

df.sort_index()

game_id,name,url
gid,,
3,X-Moon,https://leafo.itch.io/x-moon
16,Escape From Andromeda,https://adit.itch.io/escape-from-andromeda
24,DARKLORD - An Arcade Platformer,https://tyoung.itch.io/darklord---an-arcade-pl...
28,Homunculus,https://doom-crow.itch.io/homunculus
37,PEA,https://gameofmax.itch.io/pea
39,Let's Learn Japanese: Deluxe,https://gzstorm.itch.io/lets-learn-japanese-de...
41,Adlib,https://spottedzebra.itch.io/adlib
42,Planet's Core,https://2bitent.itch.io/planetscore
44,Queen Frances and the Rat Rebellion,https://manikin.itch.io/rat-rebellion


In [68]:
df['url']

gid
84296                      https://warmfeet.itch.io/thelift
131340                     https://brkeogh.itch.io/dressage
30244                       https://echowaffle.itch.io/pong
202237         https://urnext7634.itch.io/capeman-adventure
166288      https://vicleandro27.itch.io/game-usb-abc-fase1
202504          https://morvos.itch.io/procedural-city-maze
51026        https://enahs.itch.io/bazookoil-myfirstgamejam
227023                      https://mandr96.itch.io/msrtest
170697                      https://slambert.itch.io/return
108571               https://smartdevon.itch.io/hicks-quest
148495    https://mightycrazyfoo.itch.io/graffiti-block-...
111606                   https://sgx.itch.io/christmas-2015
92937        https://treechicken.itch.io/the-senpai-offense
183697     https://kai3114.itch.io/pyramid-treasure-mystery
241198    https://games-im-ashamed-of.itch.io/super-eye-...
71578     https://send2mage.itch.io/patrick-mcgaughey-du...
84662                https://thegame

In [17]:
biglist[12]

('Kingdom Hearts nobody survives. (3d)',
 'https://horrornerd.itch.io/kingdom-hearts-nobody-survives',
 73957)